## Compare results for multiple results

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [5]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_plot_spectrum_3d(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1


def f_compute_chisqr(dict_val,dict_sample,img_size):
    '''
    Compute chi-square values for sample w.r.t input images
    Input: 2 dictionaries with 4 keys for histogram and spectrum values and errors
    '''
    ### !!Both pixel histograms MUST have same bins and normalization!
    ### Compute chi-sqr
    ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
    ###  chi_sqr :: sum((Obs-Val)^2/(Val))
    
    chisqr_dict={}
    
    try: 
        val_dr=dict_val['hist_val'].copy()
        val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins

        sq_diff=(dict_val['hist_val']-dict_sample['hist_val'])**2

        size=len(dict_val['hist_val'])
        l1,l2=int(size*0.3),int(size*0.7)
        keys=['chi_1a','chi_1b','chi_1c','chi_1']
        
        for (key,start,end) in zip(keys,[0,l1,l2,0],[l1,l2,None,None]):  # 4 lists : small, medium, large pixel values and full 
            chisqr_dict.update({key:np.sum(np.divide(sq_diff[start:end],val_dr[start:end]))})

        idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
    #     chisqr_dict.update({'chi_sqr1':})

        chisqr_dict.update({'chi_2':np.sum(np.divide(sq_diff[:idx],1.0))}) ## chi-sqr without denominator division
        chisqr_dict.update({'chi_imgvar':np.sum(dict_sample['hist_err'][:idx])/np.sum(dict_val['hist_err'][:idx])}) ## measures total spread in histograms wrt to input data

        idx=img_size
        spec_diff=(dict_val['spec_val']-dict_sample['spec_val'])**2
        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec1':np.sum(spec_diff[:idx]/dict_sample['spec_val'][:idx]**2)})

        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec2':np.sum(spec_diff[:idx]/dict_sample['spec_sdev'][:idx]**2)})
        
        spec_loss=1.0*np.log(np.mean((dict_val['spec_val'][:idx]-dict_sample['spec_val'][:idx])**2))+1.0*np.log(np.mean((dict_val['spec_sdev'][:idx]-dict_sample['spec_sdev'][:idx])**2))
        chisqr_dict.update({'chi_spec3':spec_loss})
    
    except Exception as e: 
        print(e)
        
        keys=['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
        chisqr_dict=dict.fromkeys(keys,np.nan)
        pass
    
    return chisqr_dict
    

In [9]:
# img_size=64
img_size=128

In [10]:
val_data_dict={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing',
              '128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube'}

In [12]:
### Read validation data
# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)

bkgnd_dict={}
num_bkgnd=100
sigma_lst=[0.5,0.65,0.8,1.1]
labels_lst=[0,1,2,3]
### Extract validation data   
for label in labels_lst:
    fname=val_data_dict[str(img_size)]+'/norm_1_sig_{0}_train_val.npy'.format(sigma_lst[label])
    print(fname)
    samples=np.load(fname,mmap_mode='r')[-num_bkgnd:][:,0,:,:,:]
    
    dict_val=f_compute_hist_spect(samples,bins)
    bkgnd_dict[str(sigma_lst[label])]=dict_val
# del samples


/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.5_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.65_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.8_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_1.1_train_val.npy


In [49]:
df_runs=pd.DataFrame([])

In [50]:
### bkgnd: Get bkgnd chi-squares for different sigma wrt one standard sigma 
lst=[0.5,0.65,0.8,1.1]
param_label=0.65
for count,run in enumerate(lst):
    num=100
    fname=val_data_dict[str(img_size)]+'/norm_1_sig_{0}_train_val.npy'.format(run)
    print(fname)    
    samples=np.load(fname,mmap_mode='r')[1000:1000+num][:,0,:,:,:]
    dict1={'label':'ref_%s'%(run),'fname':''}
    dict1['num_imgs']=num

    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(samples,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample,img_size)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    
    df_runs=df_runs.append(dict1,ignore_index=True)


/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.5_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.65_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.8_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_1.1_train_val.npy


### Add pytorch runs

In [51]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/'
# fldr='20210617_204752_cgan_bs32_nodes8_lr0.0001-vary_fm50/'
fldr='20210628_73858_cgan_128_nodes1_lr0.0002/'
parent_dir+=fldr
param_label=0.65

# Prediction images 

for param_label in sigma_lst:
    prefix='pred_%s_'%(param_label)
    lst=[ (96, 22640), (97, 22720), (98, 23030), (101, 23830), (101, 23840), (106, 24970), (106, 24990), (106, 25000)]
#     lst=[(101, 23830), (106, 25000)]

    for count,run in enumerate(lst):
        epoch,step=run[0],run[1]
        key=prefix+'{0}-{1}'.format(epoch,step)
        print(key)
        fname=glob.glob(parent_dir+'images/'+'inference_*label-{2}_epoch-{0}_step-{1}.npy'.format(epoch,step,param_label))[0]
        dict1={'label':key,'fname':fname}
        images=np.load(dict1['fname'])     ### No transform needed here

        images=images[:,0,:,:,:]
        print(images.shape)
        print(np.max(images),np.min(images))

        dict1['num_imgs']=images.shape[0]
        ### Compute spectrum and histograms
        dict_sample=f_compute_hist_spect(images,bins)
        ### Compute chi squares
        dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample,img_size)

        dict1.update(dict_sample)
        dict1.update(dict_chisqrs)
        del(images)
        df_runs=df_runs.append(dict1,ignore_index=True)


pred_0.5_96-22640
(32, 128, 128, 128)
0.99642175 -0.9964048
pred_0.5_97-22720
(32, 128, 128, 128)
0.9969895 -0.99653405
pred_0.5_98-23030
(32, 128, 128, 128)
0.9940273 -0.996323
pred_0.5_101-23830
(32, 128, 128, 128)
0.99594975 -0.99661607
pred_0.5_101-23840
(32, 128, 128, 128)
0.997354 -0.9973607
pred_0.5_106-24970
(32, 128, 128, 128)
0.9973952 -0.99735326
pred_0.5_106-24990
(32, 128, 128, 128)
0.9962288 -0.99715215
pred_0.5_106-25000
(32, 128, 128, 128)
0.9974412 -0.9969408
pred_0.65_96-22640
(32, 128, 128, 128)
0.9984188 -0.9966568
pred_0.65_97-22720
(32, 128, 128, 128)
0.997266 -0.9981732
pred_0.65_98-23030
(32, 128, 128, 128)
0.9977491 -0.99756974
pred_0.65_101-23830
(32, 128, 128, 128)
0.9972793 -0.9979727
pred_0.65_101-23840
(32, 128, 128, 128)
0.9973454 -0.9974639
pred_0.65_106-24970
(32, 128, 128, 128)
0.9973678 -0.9982718
pred_0.65_106-24990
(32, 128, 128, 128)
0.99759257 -0.9981354
pred_0.65_106-25000
(32, 128, 128, 128)
0.9985846 -0.9982346
pred_0.8_96-22640
(32, 128, 128, 

In [52]:
# Train images 
for param_label in sigma_lst:
    prefix='train_%s_'%(param_label)
    lst=[(101, 23830), (106, 25000)]

    for count,run in enumerate(lst):
        epoch,step=run[0],run[1]
        key=prefix+'{0}-{1}'.format(epoch,step)
        print(key)
        try: 
            fname=glob.glob(parent_dir+'images/'+'gen_img_label-{2}_epoch-{0}_step-{1}.npy'.format(epoch,step,param_label))[0]
            dict1={'label':key,'fname':fname}
            images=np.load(dict1['fname'])     ### No transform needed here

            images=images[:,0,:,:,:]
            print(images.shape)
            print(np.max(images),np.min(images))

            dict1['num_imgs']=images.shape[0]
            ### Compute spectrum and histograms
            dict_sample=f_compute_hist_spect(images,bins)
            ### Compute chi squares
            dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample,img_size)

            dict1.update(dict_sample)
            dict1.update(dict_chisqrs)
            del(images)
            df_runs=df_runs.append(dict1,ignore_index=True)
        except: 
            pass

train_0.5_101-23830
(8, 128, 128, 128)
0.9955211 -0.9956417
train_0.5_106-25000
(8, 128, 128, 128)
0.9950068 -0.9942712
train_0.65_101-23830
train_0.65_106-25000
train_0.8_101-23830
(8, 128, 128, 128)
0.99657446 -0.9981975
train_0.8_106-25000
(8, 128, 128, 128)
0.9979287 -0.9977745
train_1.1_101-23830
(8, 128, 128, 128)
0.99874777 -0.9995159
train_1.1_106-25000
(8, 128, 128, 128)
0.9981636 -0.9991467


In [53]:
df_runs.columns

Index(['chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2', 'chi_imgvar',
       'chi_spec1', 'chi_spec2', 'chi_spec3', 'fname', 'hist_bin_centers',
       'hist_err', 'hist_val', 'label', 'num_imgs', 'spec_sdev', 'spec_val'],
      dtype='object')

In [54]:
col_list=['label', 'chi_1', 'chi_1a', 'chi_spec1', 'chi_spec3','chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2', 'hist_bin_centers', 'hist_err', 'hist_val','spec_sdev', 'spec_val','num_imgs','fname']

df_runs=df_runs.reindex(columns=col_list)


In [55]:
df_runs

,label,chi_1,chi_1a,chi_spec1,chi_spec3,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,hist_bin_centers,hist_err,hist_val,spec_sdev,spec_val,num_imgs,fname
0,ref_0.5,0.084855,0.057658,1.027117,58.889042,0.026169,0.001028,0.030951,0.809446,567.714923,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0026209318746665865, 0.0012307953312302102,...","[0.8475708290444763, 0.590315426330211, 0.0400...","[33061027.02429947, 9522816.482167501, 6243423...","[150054862.58426616, 82111263.64840499, 499411...",100.0,
1,ref_0.65,0.000219,0.000076,0.005281,49.566061,0.000088,0.000055,0.000021,1.006303,1.631127,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0027108935448769697, 0.0012516292039354302,...","[1.003411341364514, 0.5017591247274171, 0.0444...","[46197623.65859872, 13045425.547887238, 707084...","[182684482.71750823, 99577321.95103155, 590998...",100.0,
2,ref_0.8,0.067837,0.031967,0.447982,56.679259,0.029915,0.005955,0.019054,1.404328,431.451944,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0033773874384415577, 0.0015439952478456908,...","[1.1193565854990282, 0.4367787381814766, 0.045...","[51983649.084842816, 18467489.390514687, 93113...","[207834810.63940406, 118626816.41525161, 69767...",100.0,
3,ref_1.1,0.653081,0.163339,3.304638,62.367713,0.285926,0.203817,0.097965,1.973415,2359.262558,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0037687180020283062, 0.0016886393656873344,...","[1.2712811276325067, 0.35284926376265296, 0.04...","[79229887.83062401, 26717249.389724016, 150764...","[275027165.9278429, 149601425.99259686, 855419...",100.0,
4,pred_0.5_96-22640,0.024280,0.001591,15.955294,52.883716,0.002037,0.020653,0.000020,1.397467,12394.996558,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.003812139979841659, 0.0019662025493296675, ...","[0.8448667821111112, 0.5909983139612174, 0.043...","[28288616.231319588, 8236115.520977964, 454257...","[148831213.37602374, 85970013.28678317, 489520...",32.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
5,pred_0.5_97-22720,0.024289,0.001190,16.392133,50.769643,0.002867,0.020233,0.000211,1.325441,13573.785676,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.00321821217810746, 0.0016364725640306452, 0...","[0.8559923291798379, 0.5837388927419254, 0.045...","[34091088.01884947, 8404619.43391153, 5651204....","[146051695.9406875, 84520439.33376019, 5240394...",32.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
6,pred_0.5_98-23030,0.022126,0.001173,16.395047,51.927194,0.002667,0.018286,0.000090,1.224815,16028.609740,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0028570719692448342, 0.0014048550368018116,...","[0.8375320465486544, 0.5942126360669556, 0.044...","[33625994.983998634, 10600537.702200577, 47904...","[153901529.68701357, 86173975.82556005, 492491...",32.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
7,pred_0.5_101-23830,0.018421,0.005194,16.022604,53.669613,0.001909,0.011318,0.003484,1.036719,14909.516623,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0025568038532626093, 0.001313451054763112, ...","[0.8964631794791849, 0.5626853469462604, 0.043...","[28722403.89984013, 6902488.799130978, 3870603...","[141105230.51372015, 84090744.08621214, 511763...",32.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
8,pred_0.5_101-23840,0.016843,0.001927,16.473981,55.213268,0.001867,0.013049,0.000603,1.128604,16261.398160,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.002754913284492518, 0.0013636590195639756, ...","[0.8656647438880818, 0.5790471902470756, 0.044...","[25250742.305508073, 9613039.359396322, 563668...","[151848314.23329046, 90396965.23867826, 522352...",32.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
9,pred_0.5_106-24970,0.041051,0.008557,16.853122,51.027163,0.004939,0.027555,0.006488,1.382114,16954.961523,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.003323774072380557, 0.0017338292434854578, ...","[0.9156701535644582, 0.5527762539605315, 0.041...","[33570569.75411049, 8821694.1

### Compare chi-squares

In [56]:
chi_keys=['label','chi_1','chi_spec1', 'num_imgs', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','chi_spec3']
df_runs[chi_keys]

,label,chi_1,chi_spec1,num_imgs,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,chi_spec3
0,ref_0.5,0.084855,1.027117,100.0,0.057658,0.026169,0.001028,0.030951,0.809446,567.714923,58.889042
1,ref_0.65,0.000219,0.005281,100.0,0.000076,0.000088,0.000055,0.000021,1.006303,1.631127,49.566061
2,ref_0.8,0.067837,0.447982,100.0,0.031967,0.029915,0.005955,0.019054,1.404328,431.451944,56.679259
3,ref_1.1,0.653081,3.304638,100.0,0.163339,0.285926,0.203817,0.097965,1.973415,2359.262558,62.367713
4,pred_0.5_96-22640,0.024280,15.955294,32.0,0.001591,0.002037,0.020653,0.000020,1.397467,12394.996558,52.883716
5,pred_0.5_97-22720,0.024289,16.392133,32.0,0.001190,0.002867,0.020233,0.000211,1.325441,13573.785676,50.769643
6,pred_0.5_98-23030,0.022126,16.395047,32.0,0.001173,0.002667,0.018286,0.000090,1.224815,16028.609740,51.927194
7,pred_0.5_101-23830,0.018421,16.022604,32.0,0.005194,0.001909,0.011318,0.003484,1.036719,14909.516623,53.669613
8,pred_0.5_101-23840,0.016843,16.473981,32.0,0.001927,0.001867,0.013049,0.000603,1.128604,16261.398160,55.213268
9,pred_0.5_106-24970,0.041051,16.853122,32.0,0.008557,0.004939,0.027555,0.006488,1.382114,16954.961523,51.027163


## Plot

In [57]:
def f_plot_hist_spec(df,dict_bkg,plot_type,img_size):

    assert plot_type in ['hist','spec','spec_relative','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
#             yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            yerr2=row.spec_sdev

            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row.num_imgs)
    
            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size/2)
            plt.legend()
            plt.title("relative spectrum")
        if plot_type=='grid':
            images=np.load(row.fname)[:,0,:,:,0]
            f_plot_grid(images[:8],cols=4,fig_size=(8,4))

            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
#         y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']

        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
        plt.xlim(0,img_size/2)


    if plot_type=='spec_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
        plt.title("relative spectrum")
        plt.xlim(0,img_size/2)
        plt.ylim(0.5,2)    
    
    if plot_type in ['hist','spec']:     
        plt.yscale('log')
    plt.legend(bbox_to_anchor=(0.5, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})

    

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_runs,dict_val,'spec_relative')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [58]:
# Define the reference 
param_label=0.65
dict_val=bkgnd_dict[str(param_label)]

In [60]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type,img_size)

interact_manual(f_widget_compare,df=fixed(df_runs),
                labels_list=SelectMultiple(options=df_runs.label.values), 
                img_size=fixed(img_size),
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','spec_relative','grid']))

interactive(children=(SelectMultiple(description='labels_list', options=('ref_0.5', 'ref_0.65', 'ref_0.8', 're…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

In [45]:
col_list=['label','chi_1','chi_spec1','num_imgs','chi_2','chi_spec2']
df_runs[col_list]

,label,chi_1,chi_spec1,num_imgs,chi_2,chi_spec2
0,ref_0.5,0.086267,0.669133,200.0,0.026892,57.897684
1,ref_0.65,0.001755,0.020739,200.0,0.000171,1.399420
2,ref_0.8,0.076040,0.228422,200.0,0.020829,66.992929
3,ref_1.1,0.512256,1.663675,200.0,0.110730,284.805873
4,pred_0.5_64-37810,0.008175,8.052332,128.0,0.000093,1547.280109
5,pred_0.5_56-33080,0.015525,8.275993,128.0,0.000124,1336.722800
6,pred_0.5_58-34460,0.006913,8.416034,128.0,0.000909,1751.650775
7,pred_0.65_64-37810,0.029099,8.172498,128.0,0.001262,1289.430460
8,pred_0.65_56-33080,0.055379,8.788615,128.0,0.001885,1444.992451
9,pred_0.65_58-34460,0.043433,8.553379,128.0,0.002926,1520.402366


### Test 

In [ ]:
df_runs.fname.values

In [ ]:
# fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210112_191125_cgan_predict_1.1_m2/images/inference_spec_label-1.1_epoch-11_step-25999.npy'
# a1=np.load(fname)
# print(a1.shape)

In [ ]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_1.1_train_val.npy'
val_arr=np.load(fname)[:2000][:,0,:,:]
print(val_arr.shape)

In [ ]:
f_pixel_intensity(f_invtransform(a1),normalize=True)
plt.xscale('symlog',linthreshx=50)


# img_lst=[a1]
# f_compare_pixel_intensity(img_lst,label_lst=['a1'],bkgnd_arr=val_arr)


In [ ]:
val_arr=val_arr[:,0,:,:]

In [ ]:
img_lst=[a1,val_arr]
f_compare_spectrum(img_lst=img_lst,label_lst=['a1'],bkgnd_arr=val_arr)


### Test histogram

In [ ]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.5_train_val.npy'
print(fname)
samples=np.load(fname,mmap_mode='r')[-1000:][:,0,:,:]
print(samples.shape)